In [1]:
import pandas as pd
import numpy as np

bug_fix_factor = 1

def get_concurrency_from_SIMONA_v3(country_code, scenario, folderInputs, folderRes, folderResForStefan, fileNamingVersion):
    # Berechnung der Gleichzeitigkeit aus SOMINA-Ergebnissen
    success = False

    # Test-Werte für debugging
    country_code = 'DE0_0'
    scenario = 'sz1b'
    folderInputs = 'C:\\Users\\Workstation\\Desktop\\TNG\\sz1b\\DE0_0\\DE0_0_2022-12-06_19-52-48\\'
    folderRes = 'C:\\Users\\Workstation\\Desktop\\TNG\\sz1b\\DE0_0\\DE0_0_2022-12-06_19-52-48\\'
    fileNamingVersion = 2  # Dateien wurden umbenannt (1=_timeseries, 2=_res)

    # Other settings
    bugFixFactor = 1  # 5.111 -> Fehler wurde in SIMONA behoben
    somePlots = True

    print('#### Ermittle Gleichzeitgkeitsfunktion ########')
    print(f' # country:  {country_code}')
    print(f' # scenario: {scenario}')

    # ... Rest of the code ...
    # The remaining part of the code involves various operations including file loading, data manipulation, calculations,
    # and data visualization which would be handled separately due to its length and complexity.
    # You can use pandas for reading CSV files, numpy for array operations, and matplotlib for plotting in Python.

    return success



In [2]:
import numpy as np

# Define the time duration
T = 366 * 96
includeTime = np.ones(T, dtype=bool)

# Exclude specific time ranges
exclude_ranges = np.concatenate([np.arange(17470, 17470 + 96 * 2), np.arange(34860, T)])
includeTime[exclude_ranges] = False

# Issue a warning about the artifact at specific time points
print("Warning: There is an artifact at certain time points!")

# Now 'includeTime' is a boolean array where certain time points are marked to be excluded


In [3]:
# Define GZG parameters
GZG = {
    'P_LP': 11,
    'N_MC': 2000,
    'Kurve': [1, 2, 4, 6, 8, 10, 15, 20, 30, 50, 100, 150, 200],
    'cases': {
        1: {'EV': True, 'HP': True, 'PV': True},
        2: {'EV': True, 'HP': False, 'PV': True},
        3: {'EV': False, 'HP': True, 'PV': True},
        4: {'EV': True, 'HP': True, 'PV': False},
        5: {'EV': True, 'HP': False, 'PV': False},
        6: {'EV': False, 'HP': True, 'PV': False}
    }
}


In [4]:
# List of file names
file_names = [
    'hp_res.csv',
    'hp_input.csv',
    'evcs_res.csv',
    'evcs_input.csv',
    'pv_res.csv',
    'pv_input.csv',
    'em_input.csv',
    'load_res.csv',
    'load_input.csv',
    'storage_res.csv',
    'storage_input.csv'
]


In [5]:
import os
import pandas as pd

# Get the folder path where the files are located
folder_inputs = r'C:\tmp\20231116_flex_minigrid_2023-11-17_07-18-57\input\grid'

# Filenames
file_names = [
    'em_input.csv',
    'evcs_input.csv',
    'hp_input.csv',
    'pv_input.csv',
    'load_input.csv',
    'storage_input.csv'
]

# Read input data
print('Input data loading')

data_input_em = pd.read_csv(os.path.join(folder_inputs,file_names[0]),index_col='uuid')


# Remove em-system
data_input_em = data_input_em[~data_input_em['id'].str.contains('em-system')]

data_input_ev = pd.read_csv(os.path.join(folder_inputs, file_names[1]),index_col='uuid')
data_input_hp = pd.read_csv(os.path.join(folder_inputs, file_names[2]),index_col='uuid')
data_input_pv = pd.read_csv(os.path.join(folder_inputs, file_names[3]),index_col='uuid')
data_input_lo = pd.read_csv(os.path.join(folder_inputs, file_names[4]),index_col='uuid')
data_input_bs = pd.read_csv(os.path.join(folder_inputs, file_names[5]),index_col='uuid')
data_input_em

Input data loading


,id,node,connected_assets,control_strategy,q_characteristics,operates_from,operates_until,operator
uuid,,,,,,,,
f612dd58-4ce4-4ec4-9d46-459b3a9428b9,House_15_1,e3c3c6a3-c383-4dbb-9b3f-a14125615386,b8ed6a33-d7dc-4cf4-8a31-6e90427dff1f 8f6399a1-...,self_optimization,"cosPhiFixed:{(0.0, 0.9)}",NaN,NaN,NaN
40eec983-ecff-41df-ba23-138bc9005231,House_15_2,e3c3c6a3-c383-4dbb-9b3f-a14125615386,3dc576df-9533-4f32-8070-32c330f2d49f 0004cba4-...,self_optimization,"cosPhiFixed:{(0.0, 0.9)}",NaN,NaN,NaN
d1094dec-1ec1-4a8d-841f-d962508e41a2,House_15_3,e3c3c6a3-c383-4dbb-9b3f-a14125615386,54777c1f-c402-4e6c-9e72-5aa992ba8f53 00652030-...,self_optimization,"cosPhiFixed:{(0.0, 0.9)}",NaN,NaN,NaN
a24b4c58-6594-4968-98e7-9c2b0393a8f6,House_15_4,e3c3c6a3-c383-4dbb-9b3f-a14125615386,bb57540c-417d-4b35-aceb-0b2d4641b5bf 850e12fb-...,self_optimization,"cosPhiFixed:{(0.0, 0.9)}",NaN,NaN,NaN
0a234b30-4469-4084-a2bc-748f71a40589,House_15_5,e3c3c6a3-c383-4dbb-9b3f-a14125615386,2ac407c1-f52a-4160-bb16-24c2e873310d 105837c7-...,self_optimization,"cosPhiFixed:{(0.0, 0.9)}",NaN,NaN,NaN
d77d5329-4223-4dd2-9d26-c8e5a9245f9f,House_15_6,e3c3c6a3-c383-4dbb-9b3f-a14125615386,53cf250b-6829-4d70-98ef-2bd5f32c7deb e854a393-...,self_optimization,"cosPhiFixed:{(0.0, 0.9)}",NaN,NaN,NaN
e670a1ea-777f-4118-9c46-eaf3d3b58644,House_15_7,e3c3c6a3-c383-4dbb-9b3f-a14125615386,96e87d12-cd60-4c9a-bd5c-15fd13039e37 238b32cf-...,self_optimization,"cosPhiFixed:{(0.0, 0.9)}",NaN,NaN,NaN
3f7828fb-c268-4acd-b936-ca2fb12a51b2,House_15_8,e3c3c6a3-c383-4dbb-9b3f-a14125615386,2637d1ae-bbd2-4122-8845-38df9b180ac6 2ac889c0-...,self_optimization,"cosPhiFixed:{(0.0, 0.9)}",NaN,NaN,NaN
9b0c37e8-5ec7-433c-839f-aaeb2bfa6233,House_15_9,e3c3c6a3-c383-4dbb-9b3f-a14125615386,8513fe88-4b30-4ebe-bbc3-fe21a424a902 9b83a965-...,self_optimization,"cosPhiFixed:{(0.0, 0.9)}",NaN,NaN,NaN


In [6]:

def create_em_to_systemparticipant_dict(em_list, system_participants_list):
    
    matching_uuids_dict = dict()
    
    for index, row in system_participants_list.iterrows():
        for em_index, em_row in em_list.iterrows():
            connected_assets = em_row['connected_assets']
            
            connected_assets_list = connected_assets.strip('[]').replace("'", "").split(', ') if isinstance(connected_assets, str) else None
                       
            if isinstance(connected_assets_list, list) and index in connected_assets:
                if em_index not in matching_uuids_dict:
                    matching_uuids_dict[em_index] = index
                else:
                    # If there are multiple occurrences, store them as a list
                    if not isinstance(matching_uuids_dict[em_index], list):
                        raise ValueError("uuid is included more then once in em")
    return matching_uuids_dict
    
emToHp = create_em_to_systemparticipant_dict(data_input_em,data_input_hp)    
emToEv = create_em_to_systemparticipant_dict(data_input_em,data_input_ev)
emToBs = create_em_to_systemparticipant_dict(data_input_em,data_input_bs)
emToPv = create_em_to_systemparticipant_dict(data_input_em,data_input_pv)
emToLo = create_em_to_systemparticipant_dict(data_input_em,data_input_lo)




In [14]:
import numpy as np

# FIXME: Parsing of EVCS Charing power
if 0 == 1:
    data_input_ev['pSum'] = np.zeros(len(data_input_ev))
    for i in range(len(data_input_ev)):
        tmp = data_input_ev['type'][i].split(' ')
        data_input_ev.at[i, 'pSum'] = float(tmp[0])
    
    data_input_em['nChargePoints'] = np.zeros(len(data_input_em))
    for i in range(len(data_input_em)):
        if EM_to_ev[i]:
            ev_indices = EM_to_ev[i]
            data_input_em.at[i, 'nChargePoints'] = data_input_ev.loc[ev_indices, 'charging_points'].sum()


In [15]:
import csv
import datetime

def read_sort_time_series(filename, data_input):
     # Read the CSV file
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        hdr = next(reader)
        num = len(hdr) - 1  # Number of columns
        
        # Interpret timestamp
        timestamp = [datetime.strptime(row[0], '%Y-%m-%d %H:%M:%S') for row in reader]

    # Get UnitIDs
    unitIDs = hdr[1:]
    nUnits = len(unitIDs)

    # Read data and handle exceptions
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header
        try:
            time_series = [[float(val) for val in row[1:]] for row in reader]
        except ValueError:
            data = list(reader)
            L = [len(row) - 1 for row in data]
            time_series = []
            for i, row in enumerate(data):
                vals = [float(val) if val else float('nan') for val in row[1:]]
                vals.extend([float('nan')] * (max(L) - L[i]))
                time_series.append(vals)

    # Adjust sorting based on input data
    time_series_sort = [[float('nan') for _ in range(nUnits)] for _ in range(len(time_series))]
    for i in range(nUnits):
        for j, uuid in enumerate(inputs['uuid']):
            if uuid in unitIDs:
                xx = unitIDs.index(uuid)
                if xx < nUnits:
                    time_series_sort[:, i] = time_series[:, xx]

    return time_series_sort

In [16]:
import pandas as pd
import numpy as np
import os
import csv
from datetime import datetime
from pypsdm.processing.series import (
    add_series,)
from pypsdm.models.result import (
    entity,)
from pypsdm.models import (
    enums,)


folder_res = r'C:\tmp\20231116_flex_minigrid_2023-11-17_07-18-57\rawOutputData'
filename_hp = "hp_res.csv"
filename_ev = "evcs_res.csv"
filename_pv = "pv_res.csv"
filename_lo = "load_res.csv"
filename_bs = "storage_res.csv"

filename = os.path.join(folder_res, filename_hp)   
     
timeseries_data = pd.read_csv(os.path.join(folder_res, filename_hp))
#print(timeseries_data)

#print(len(timeseries_data.loc[1,:].values[4]))



# Function to add seconds if they're missing from the timestamp
# Function to add seconds if they're missing from the timestamp
def add_seconds(timestamp):
    if len(timestamp) == 22 and timestamp.endswith('Z[UTC]'):  # Check if seconds are missing and ends with 'Z[UTC]'
        timestamp = timestamp[:-6] + ':00' + timestamp[-6:]  # Insert ':00' before 'Z[UTC]'
    return timestamp


# Add seconds to all timestamps
timeseries_data['time'] = timeseries_data['time'].apply(add_seconds)

# Convert 'time' column to datetime format with optional seconds
timeseries_data['time'] = pd.to_datetime(timeseries_data['time'], format='%Y-%m-%dT%H:%M:%SZ[UTC]')

# Format 'time' column to the desired string format
timeseries_data['time'] = timeseries_data['time'].dt.strftime('%Y-%m-%d %H:%M:%S')

#specific_model = 'd3413988-ebcd-4aff-b5b5-d70364a3b7a2'  # Replace this with the specific entry you want to filter for
#filtered_df = timeseries_data[timeseries_data['input_model'] == specific_model]
#filtered_df.set_index(['time','input_model'],inplace=True)
timeseries_data.set_index(['time','input_model'],inplace=True)



endtime = datetime(2020,1,1)

#d34=entity.ResultEntities.build(enums.SystemParticipantsEnum.HEAT_PUMP,specific_model, timeseries_data, endtime)
#print(d34)



In [17]:
from pypsdm.models.input.container.participants import SystemParticipantsContainer
from pypsdm.models.result.container.participants import ParticipantsResultContainer
from pypsdm.models import gwr

gwr = gwr.GridWithResults.from_csv('flex_minigrid', folder_inputs,',',folder_res,',', simulation_end=endtime)


In [18]:
gwr.results.participants.hps

PQResultDict(entity_type=<SystemParticipantsEnum.HEAT_PUMP: 'hp'>, entities={'068e9b3d-1a8c-4270-a714-e08b23f54908': PQResult(entity_type=<SystemParticipantsEnum.HEAT_PUMP: 'hp'>, input_model='068e9b3d-1a8c-4270-a714-e08b23f54908', name='HP_10_House_15_10', data=                       p    q  q_dot
time                                
2019-01-01 00:00:00  0.0  0.0    0.0
2019-01-01 00:15:00  0.0  0.0    0.0
2019-01-01 00:30:00  0.0  0.0    0.0
2019-01-01 00:45:00  0.0  0.0    0.0
2019-01-01 01:00:00  0.0  0.0    0.0
...                  ...  ...    ...
2019-12-31 23:00:00  0.0  0.0    0.0
2019-12-31 23:15:00  0.0  0.0    0.0
2019-12-31 23:30:00  0.0  0.0    0.0
2019-12-31 23:45:00  0.0  0.0    0.0
2020-01-01 00:00:00  0.0  0.0    0.0

[35717 rows x 3 columns]), '2538e78e-1ce9-4291-b43f-3cab3746e4c2': PQResult(entity_type=<SystemParticipantsEnum.HEAT_PUMP: 'hp'>, input_model='2538e78e-1ce9-4291-b43f-3cab3746e4c2', name='HP_11_House_15_21', data=                       p    q  q_dot
time 

In [19]:

if 0 == 1:
    
    #p_ref_sum = add_series(self.p_ref(), other.p_ref(), "p_ref")
    #series = add_series()
    
    print(test)
    print(' Zeitreihen HP laden')
    #time_series_HP = read_sort_time_series(os.path.join(folder_res, filename_hp), data_input_hp)
    
    print(' Zeitreihen EV laden')
    #time_series_EV = read_sort_time_series(folder_res + filename_ev, data_input_EV)
    
    print(' Zeitreihen PV laden')
    #time_series_PV = read_sort_time_series(folder_res + filename_pv, data_input_PV)
    
    print(' Zeitreihen Lo laden')
    #time_series_Lo = read_sort_time_series(folder_res + filename_lo, data_input_Lo)
    
    print(' Zeitreihen BS laden')
    #time_series_BS = read_sort_time_series(folder_res + filename_bs, data_input_BS)

if 0 == 1:
    # Check for mismatched lengths and pad if needed
    l_max = max(len(time_series_HP), len(time_series_EV), len(time_series_PV), len(time_series_Lo), len(time_series_BS))
    
    def pad_time_series(time_series, l_max):
        d = l_max - len(time_series)
        return np.vstack([time_series, np.zeros((d, time_series.shape[1]))])
    
    if len(time_series_HP) < l_max:
        time_series_HP = pad_time_series(time_series_HP, l_max)
    
    if len(time_series_EV) < l_max:
        time_series_EV = pad_time_series(time_series_EV, l_max)
    
    if len(time_series_PV) < l_max:
        time_series_PV = pad_time_series(time_series_PV, l_max)
    
    if len(time_series_Lo) < l_max:
        time_series_Lo = pad_time_series(time_series_Lo, l_max)
    
    if len(time_series_BS) < l_max:
        time_series_BS = pad_time_series(time_series_BS, l_max)


In [20]:
import numpy as np
import pandas as pd

N = dataInput_EM.shape[0]
T = timeSeries_EV.shape[0]

Pool = {
    "EM_pSum": np.zeros((T, N)),
    "EM_pScal": np.zeros((T, N)),
    "BS": np.zeros((T, N)),
    "inst_EV": np.zeros(N),
    "inst_HP": np.zeros(N),
    "inst_PV": np.zeros(N),
    "inst_BS": np.zeros(N),
    "EM_nurLoad_pScal": np.zeros((T, N)),
    "all": np.zeros((T, N)),
    "pv": np.zeros((T, N)),
    "ev": np.zeros((T, N)),
    "ev2": np.zeros((T, N)),
}

for i in range(N):
    pv = np.zeros(T)
    inst_pv = 0
    if EM_toPV[i]:
        pv = timeSeries_PV[:, EM_toPV[i]] * 1e3 / dataInput_EM.nBuildings[i]
        inst_pv = -np.min(pv)
    
    ev = np.zeros(T)
    inst_ev = 0
    if EM_toEV[i]:
        if EM_toEV[i] <= timeSeries_EV.shape[1]:
            ev = timeSeries_EV[:, EM_toEV[i]] / np.max(timeSeries_EV[:, EM_toEV[i]])
            inst_ev = dataInput_EV.charging_points[EM_toEV[i]] * 11  # kW
            ev *= inst_ev
            ev[ev < 0] = 0
            ev[np.isnan(ev)] = 0

    hp = np.zeros(T)
    inst_hp = 0
    if EM_toHP[i]:
        hp = timeSeries_HP[:, EM_toHP[i]] * 1e3 / dataInput_EM.nBuildings[i]
        inst_hp = np.max(hp)
    
    if EM_toLo[i]:
        lo = timeSeries_Lo[:, EM_toLo[i]] * 1e3 / dataInput_EM.nBuildings[i]
    else:
        lo = np.zeros(T)
    
    bs = np.zeros(T)
    inst_bs = 0
    if EM_toBS[i]:
        if EM_toBS[i] <= timeSeries_BS.shape[1]:
            bs = timeSeries_BS[:, EM_toBS[i]] * 1e3 / dataInput_EM.nBuildings[i]
            inst_bs = np.max(np.abs(bs))
        else:
            print(f"Warning: Zeitreihe BS #{EM_toBS[i]} nicht vorhanden")
    
    pvSurplus = pv + lo
    pvSurplus[pvSurplus > 0] = 0
    hp_ev_bs_pv_sum = ev + hp + bs + pvSurplus
    hp_ev_bs_pv_sum[hp_ev_bs_pv_sum < 0] = 0
    hp_ev_sum = ev + hp

    Pool["EM_pSum"][:, i] = hp_ev_bs_pv_sum
    Pool["BS"][:, i] = bs
    if (inst_ev + inst_hp) > 0:
        Pool["EM_pScal"][:, i] = hp_ev_bs_pv_sum / (inst_ev + inst_hp)
        Pool["EM_nurLoad_pScal"][:, i] = hp_ev_sum / (inst_ev + inst_hp)
    else:
        Pool["EM_pScal"][:, i] = 0
        Pool["EM_nurLoad_pScal"][:, i] = 0
    
    Pool["inst_EV"][i] = inst_ev
    Pool["inst_HP"][i] = inst_hp
    Pool["inst_PV"][i] = inst_pv
    Pool["inst_BS"][i] = inst_bs
    Pool["all"][:, i] = lo + pv + bs + hp + ev
    Pool["pv"][:, i] = pv
    Pool["ev"][:, i] = ev
    Pool["ev2"][:, i] = ev2

    # Plotting section can be implemented here if required

# The plotting section from the original MATLAB code can be added similarly in Python for visualization purposes.
# Ensure the necessary plotting libraries like Matplotlib are imported.


NameError: name 'dataInput_EM' is not defined

In [21]:
# Vorhandene Fälle
# PV immer an
# 1. Spalte: BS
# 2. Spalte: EV
# 3. Spalte: HP vorhanden
# 4. Spalte: neue HP & Dämmung


cases = [
    (1, 1, 1, 1),
    (1, 1, 1, 0),
    (1, 1, 0, 1),
    (1, 1, 0, 0),
    (1, 0, 1, 1),
    (1, 0, 1, 0),
    (1, 0, 0, 1),
    (1, 0, 0, 0),
    (0, 1, 1, 1),
    (0, 1, 1, 0),
    (0, 1, 0, 1),
    (0, 1, 0, 0),
    (0, 0, 1, 1),
    (0, 0, 1, 0),
    (0, 0, 0, 1),
    (0, 0, 0, 0),
]

# Resultierende Fälle für GLZ => Remove wenn 3. Spalte == 1 und Spalte 1 und 2 nicht 1

filtered_cases = [case for case in cases if case[2] != 0 or case[0] == 1 or case[1] == 1]

filtered_cases

# Fälle Binär-codiert (1,1,1,1 ==15)
# Gemeinsame Analyse von 
# Fall 14 und Fall 15 (identisch nur neue oder alte WP & Dämmung) => BS + EV + WP
# 13 und 12 (identisch) => BS + EV
# 11 und 10 (identisch nur neue oder alte WP & Dämmung) BS + WP 
# 9 und 8 (identisch) => Nur BS
# 7 und 6 (identisch nur neue oder alte WP & Dämmung) => EV + WP 
# 4 und 5 (identisch) => Nur EV
# 2 und 3 (identisch nur neue oder alte WP & Dämmung) => Nur WP




[(1, 1, 1, 1),
 (1, 1, 1, 0),
 (1, 1, 0, 1),
 (1, 1, 0, 0),
 (1, 0, 1, 1),
 (1, 0, 1, 0),
 (1, 0, 0, 1),
 (1, 0, 0, 0),
 (0, 1, 1, 1),
 (0, 1, 1, 0),
 (0, 1, 0, 1),
 (0, 1, 0, 0),
 (0, 0, 1, 1),
 (0, 0, 1, 0)]

In [22]:

folder_glz_cases = r'C:\tmp\20231116_flex_minigrid_2023-11-17_07-18-57\input\GLZ'
file_name_glz_cases = 'em_to_case_dict.csv'
em_cases_dict = pd.read_csv(os.path.join(folder_glz_cases, file_name_glz_cases), index_col=0).rename_axis(index='em_uuid')

em_cases_dict

,0,1,2,3
em_uuid,,,,
849c3aad-9758-4e7f-bb19-926d6576249d,1,1,1,1
5341ea74-bd3e-4667-aa86-9b227f14fc40,1,1,1,1
331583c0-dcc8-4fd4-af61-43edd552d522,1,1,1,1
92ab4409-1787-41e4-ad0f-1b0472d6caf4,1,1,1,1
06ddcddf-6775-4f89-9829-698e1c414417,1,1,1,1
fffd90ac-bb39-4db9-91f0-538d6c68326b,1,1,1,0
5f892d13-8b42-4ca6-a8c3-c421d0bb5518,1,1,1,0
ab8e8f94-7e8e-4fc1-978e-168dace4521b,1,1,1,0
ae0cca45-993d-473f-bb1c-21db84ec8933,1,1,1,0


In [23]:
ems_grid = gwr.grid.participants.ems
ems_results=gwr.results.participants.ems

test_uuid = 'f9592ef6-1fbf-43e4-9268-98daac369d66'

ems_results.get(test_uuid).data





,p,q
time,,
2019-01-01 00:00:00,0.000844,0.000409
2019-01-01 00:15:00,0.000759,0.000368
2019-01-01 00:30:00,0.000686,0.000332
2019-01-01 00:45:00,0.000623,0.000302
2019-01-01 01:00:00,0.000577,0.000279
...,...,...
2019-12-31 23:00:00,0.001238,0.000600
2019-12-31 23:15:00,0.001140,0.000552
2019-12-31 23:30:00,0.001042,0.000505


In [24]:
gwr.grid.participants.ems.data





,id,node,connected_assets,control_strategy,q_characteristics,operates_from,operates_until,operator
uuid,,,,,,,,
f612dd58-4ce4-4ec4-9d46-459b3a9428b9,House_15_1,e3c3c6a3-c383-4dbb-9b3f-a14125615386,"[b8ed6a33-d7dc-4cf4-8a31-6e90427dff1f, 8f6399a...",self_optimization,"cosPhiFixed:{(0.0, 0.9)}",NaN,NaN,NaN
40eec983-ecff-41df-ba23-138bc9005231,House_15_2,e3c3c6a3-c383-4dbb-9b3f-a14125615386,"[3dc576df-9533-4f32-8070-32c330f2d49f, 0004cba...",self_optimization,"cosPhiFixed:{(0.0, 0.9)}",NaN,NaN,NaN
d1094dec-1ec1-4a8d-841f-d962508e41a2,House_15_3,e3c3c6a3-c383-4dbb-9b3f-a14125615386,"[54777c1f-c402-4e6c-9e72-5aa992ba8f53, 0065203...",self_optimization,"cosPhiFixed:{(0.0, 0.9)}",NaN,NaN,NaN
a24b4c58-6594-4968-98e7-9c2b0393a8f6,House_15_4,e3c3c6a3-c383-4dbb-9b3f-a14125615386,"[bb57540c-417d-4b35-aceb-0b2d4641b5bf, 850e12f...",self_optimization,"cosPhiFixed:{(0.0, 0.9)}",NaN,NaN,NaN
0a234b30-4469-4084-a2bc-748f71a40589,House_15_5,e3c3c6a3-c383-4dbb-9b3f-a14125615386,"[2ac407c1-f52a-4160-bb16-24c2e873310d, 105837c...",self_optimization,"cosPhiFixed:{(0.0, 0.9)}",NaN,NaN,NaN
d77d5329-4223-4dd2-9d26-c8e5a9245f9f,House_15_6,e3c3c6a3-c383-4dbb-9b3f-a14125615386,"[53cf250b-6829-4d70-98ef-2bd5f32c7deb, e854a39...",self_optimization,"cosPhiFixed:{(0.0, 0.9)}",NaN,NaN,NaN
e670a1ea-777f-4118-9c46-eaf3d3b58644,House_15_7,e3c3c6a3-c383-4dbb-9b3f-a14125615386,"[96e87d12-cd60-4c9a-bd5c-15fd13039e37, 238b32c...",self_optimization,"cosPhiFixed:{(0.0, 0.9)}",NaN,NaN,NaN
3f7828fb-c268-4acd-b936-ca2fb12a51b2,House_15_8,e3c3c6a3-c383-4dbb-9b3f-a14125615386,"[2637d1ae-bbd2-4122-8845-38df9b180ac6, 2ac889c...",self_optimization,"cosPhiFixed:{(0.0, 0.9)}",NaN,NaN,NaN
9b0c37e8-5ec7-433c-839f-aaeb2bfa6233,House_15_9,e3c3c6a3-c383-4dbb-9b3f-a14125615386,"[8513fe88-4b30-4ebe-bbc3-fe21a424a902, 9b83a96...",self_optimization,"cosPhiFixed:{(0.0, 0.9)}",NaN,NaN,NaN


In [76]:
def getEmInstalledCapacatiyFromUuid_BACKUP(em_uuid, ems_grid):
    #em_uuid = 'f612dd58-4ce4-4ec4-9d46-459b3a9428b9'
    
    
    
    load_srated=0
    hp_srated=0
    evcs_srated=0
    bs_srated=0
    pv_srated=0
    total_s_rated_em_load_direction = 0
    total_s_rated_em_feedin_direction = 0
    em_installed_capacity = pd.DataFrame(columns=['s_rated_em_load_direction', 's_rated_em_feedin_direction'])
    for em in ems_grid.uuid:
        for conected_asset in ems_grid.connected_assets.get(em):
            if gwr.grid.participants.loads.__contains__(conected_asset):
                load_srated=gwr.grid.participants.loads.get(conected_asset)['s_rated']     
            if gwr.grid.participants.hps.__contains__(conected_asset):
                hp_srated=gwr.grid.participants.hps.get(conected_asset)['s_rated']
            if gwr.grid.participants.evcs.__contains__(conected_asset):
                evcs_srated=gwr.grid.participants.evcs.get(conected_asset)['power']
            if gwr.grid.participants.storages.__contains__(conected_asset):
                bs_srated=gwr.grid.participants.storages.get(conected_asset)['s_rated']
            if gwr.grid.participants.pvs.__contains__(conected_asset):
                pv_srated=gwr.grid.participants.pvs.get(conected_asset)['s_rated']
            
    ##### TODO FIXME: LOAD INCLUDED HERE OR NOT????????????????????????????????????????????????????????????????????
            s_rated_em_load_direction = load_srated + hp_srated + evcs_srated + bs_srated
            s_rated_em_feedin_direction = evcs_srated + bs_srated + pv_srated
            em_installed_capacity.loc[em]= [s_rated_em_load_direction,s_rated_em_feedin_direction]
                
    return em_installed_capacity
    
    


In [111]:
def getEmInstalledCapacatiyFromUuid(em_uuid, ems_grid):

    load_srated=0
    hp_srated=0
    evcs_srated=0
    bs_srated=0
    pv_srated=0
    total_s_rated_em_load_direction = 0
    total_s_rated_em_feedin_direction = 0
    em_installed_capacity = pd.DataFrame(columns=['s_rated_em_load_direction', 's_rated_em_feedin_direction'])   
    
    for conected_asset in ems_grid.connected_assets.get(em_uuid):
        if gwr.grid.participants.loads.__contains__(conected_asset):
            load_srated=gwr.grid.participants.loads.get(conected_asset)['s_rated']     
        if gwr.grid.participants.hps.__contains__(conected_asset):
            hp_srated=gwr.grid.participants.hps.get(conected_asset)['s_rated']
        if gwr.grid.participants.evcs.__contains__(conected_asset):
            evcs_srated=gwr.grid.participants.evcs.get(conected_asset)['power']
        if gwr.grid.participants.storages.__contains__(conected_asset):
            bs_srated=gwr.grid.participants.storages.get(conected_asset)['s_rated']
        if gwr.grid.participants.pvs.__contains__(conected_asset):
            pv_srated=gwr.grid.participants.pvs.get(conected_asset)['s_rated']
            
    ##### TODO FIXME: LOAD INCLUDED HERE OR NOT????????????????????????????????????????????????????????????????????
    s_rated_em_load_direction = load_srated + hp_srated + evcs_srated + bs_srated
    s_rated_em_feedin_direction = evcs_srated + bs_srated + pv_srated
    em_installed_capacity.loc[em]= [s_rated_em_load_direction,s_rated_em_feedin_direction]
                
    return em_installed_capacity

In [112]:
print(em_cases_dict)

                                      0  1  2  3
em_uuid                                         
849c3aad-9758-4e7f-bb19-926d6576249d  1  1  1  1
5341ea74-bd3e-4667-aa86-9b227f14fc40  1  1  1  1
331583c0-dcc8-4fd4-af61-43edd552d522  1  1  1  1
92ab4409-1787-41e4-ad0f-1b0472d6caf4  1  1  1  1
06ddcddf-6775-4f89-9829-698e1c414417  1  1  1  1
fffd90ac-bb39-4db9-91f0-538d6c68326b  1  1  1  0
5f892d13-8b42-4ca6-a8c3-c421d0bb5518  1  1  1  0
ab8e8f94-7e8e-4fc1-978e-168dace4521b  1  1  1  0
ae0cca45-993d-473f-bb1c-21db84ec8933  1  1  1  0
1830af2f-4cf4-490e-86d7-91b2bfddc04d  1  1  1  0
00071083-835f-4d3f-b95a-dba4184a026b  1  1  0  1
2976b2cc-c4bf-4ebb-b7f3-4da10d488406  1  1  0  1
7429f380-7dbe-493b-8340-210dd3b11c38  1  1  0  1
95db9dce-e18f-4d48-b7ad-4ba94e0fe231  1  1  0  1
e2a2584a-53e2-47f3-b5fa-11034518d966  1  1  0  1
67880654-8805-4a14-8d2a-c97104a799f9  1  1  0  0
d8959281-8895-4ca3-a6d8-eedadc62e427  1  1  0  0
b50bc551-cbb1-46af-9c1a-63f43bc44226  1  1  0  0
dc0199b4-6c8f-4e29-8

In [113]:
def getCasesFromConditions(dict, cond1, cond2, cond3, cond4):
    filtered_uuids = []
    
    # Loop through the dictionary items and filter based on conditions
    for uuid, values in dict.iterrows():
        if values[0] == cond1 and values[1] == cond2 and values[2] == cond3 and values[3] == cond4:
            filtered_uuids.append(uuid)
    return  filtered_uuids


In [114]:
test = getCasesFromConditions(em_cases_dict, 1,1,1,1)
print(test)

['849c3aad-9758-4e7f-bb19-926d6576249d', '5341ea74-bd3e-4667-aa86-9b227f14fc40', '331583c0-dcc8-4fd4-af61-43edd552d522', '92ab4409-1787-41e4-ad0f-1b0472d6caf4', '06ddcddf-6775-4f89-9829-698e1c414417']


In [116]:
getEmInstalledCapacatiyFromUuid('f612dd58-4ce4-4ec4-9d46-459b3a9428b9', ems_grid)

,s_rated_em_load_direction,s_rated_em_feedin_direction
032b9074-d561-449c-8bd1-6d6b6c23f8e5,39.605514,43.124623


In [81]:
for item in test:
    val = getEmInstalledCapacatiyFromUuid(item, ems_grid)
    print(val)

                                      s_rated_em_load_direction  \
f612dd58-4ce4-4ec4-9d46-459b3a9428b9                  39.605514   
40eec983-ecff-41df-ba23-138bc9005231                  36.498822   
d1094dec-1ec1-4a8d-841f-d962508e41a2                  41.601886   
a24b4c58-6594-4968-98e7-9c2b0393a8f6                  40.012344   
0a234b30-4469-4084-a2bc-748f71a40589                  36.439804   
d77d5329-4223-4dd2-9d26-c8e5a9245f9f                  36.694908   
e670a1ea-777f-4118-9c46-eaf3d3b58644                  38.109403   
3f7828fb-c268-4acd-b936-ca2fb12a51b2                  33.134497   
9b0c37e8-5ec7-433c-839f-aaeb2bfa6233                  35.736906   
3f11ce27-006d-4237-9922-4524577e71b6                  33.057605   
6b2e3229-3d37-4c21-acf4-95e94d5eeb80                  33.854947   
5134b6e8-931a-4349-813b-88e1fbe21820                  35.201258   
5ba62ac4-1c90-4b4d-8ddd-a00ccc0f546b                  35.295169   
1416998d-aec1-4397-b284-4d6b70bf1753                  34.39807

In [92]:
ems_grid.get('f612dd58-4ce4-4ec4-9d46-459b3a9428b9')

id                                                          House_15_1
node                              e3c3c6a3-c383-4dbb-9b3f-a14125615386
connected_assets     [b8ed6a33-d7dc-4cf4-8a31-6e90427dff1f, 8f6399a...
control_strategy                                     self_optimization
q_characteristics                             cosPhiFixed:{(0.0, 0.9)}
operates_from                                                      NaN
operates_until                                                     NaN
operator                                                           NaN
Name: f612dd58-4ce4-4ec4-9d46-459b3a9428b9, dtype: object

In [ ]:
tmp_hp_srated=gwr.grid.participants.evcs.get(asset)['s_rated']

print(tmp_hp_srated)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
if 0 == 1:
    GZG_N_Kurve = len(GZG_Kurve)
    GZG_nSamples = np.zeros(GZG_nCases)
    GZG_nBuildings = np.zeros(GZG_nCases)
    GZG_pInstTotal = np.zeros(GZG_nCases)
    
    # ... (initialize other variables)
    
    for c in range(1, GZG_nCases + 1):
        try:
            # ... (code within the loop)
    
            fltrEV = np.logical_and(Pool_inst_EV > 0, dataInput_EM.nChargePoints == 1) if GZG_cases[c]['EV'] else Pool_inst_EV == 0
            fltrHP = Pool_inst_HP > 0 if GZG_cases[c]['HP'] else Pool_inst_HP == 0
            fltrPV = Pool_inst_PV > 0 if GZG_cases[c]['PV'] else Pool_inst_PV == 0
    
            fltr = np.logical_and(np.logical_and(fltrEV, fltrHP), fltrPV)
            Pool_MC = Pool_EM_pScal[:, fltr]
            GZG_nSamples[c - 1] = np.sum(fltr)
            GZG_nBuildings[c - 1] = np.sum(dataInput_EM.nBuildings[fltr])
    
            # ... (rest of the code within the loop)
    
        except Exception as e:
            print(e)
    
    print("Okay")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming GZG, Pool, and other relevant variables are defined appropriately





GZG_N_Kurve = len(GZG_Kurve)
GZG_nSamples = np.zeros(GZG_nCases)
GZG_nBuildings = np.zeros(GZG_nCases)
GZG_pInstTotal = np.zeros(GZG_nCases)

GZG_MW = np.zeros((GZG_N_Kurve, GZG_nCases))
GZG_MW_Q90 = np.zeros((GZG_N_Kurve, GZG_nCases))
GZG_MW_Q95 = np.zeros((GZG_N_Kurve, GZG_nCases))
GZG_MW_Q99 = np.zeros((GZG_N_Kurve, GZG_nCases))
GZG_MW_Qmax = np.zeros((GZG_N_Kurve, GZG_nCases))
GZG_MW_Qmed = np.zeros((GZG_N_Kurve, GZG_nCases))

for c in range(1, GZG['nCases'] + 1):
    try:
        # Filter types
        fltrEV = (Pool['inst_EV'] > 0) & (dataInput_EM['nChargePoints'] == 1) if GZG['cases'][c]['EV'] else (Pool['inst_EV'] == 0)
        fltrHP = (Pool['inst_HP'] > 0) if GZG['cases'][c]['HP'] else (Pool['inst_HP'] == 0)
        fltrPV = (Pool['inst_PV'] > 0) if GZG['cases'][c]['PV'] else (Pool['inst_PV'] == 0)

        fltr = fltrEV & fltrHP & fltrPV
        Pool_MC = Pool['EM_pScal'][:, fltr]
        GZG['nSamples'][c] = np.sum(fltr)
        GZG['nBuildings'][c] = np.sum(dataInput_EM['nBuildings'][fltr])

        GZG['pInstTotal'][c] = (np.sum(Pool['inst_EV'][fltr]) / np.sum(fltr)) * GZG['nBuildings'][c] + \
                                (np.sum(Pool['inst_HP'][fltr]) / np.sum(fltr)) * GZG['nBuildings'][c]

        temp_GG_norm = np.zeros(GZG['N_MC'])
        GZG['N_Pool'] = Pool_MC.shape[1]

        if somePlots:
            plt.subplot(2, 3, c)
            plt.title(f'{c} - {GZG["nBuildings"][c] / 1e3:.1f} Tsd.')
            plt.xlim([0, max(GZG['Kurve'])])

            # Other plot settings...

        temp_Kurve = GZG['N_Kurve']
        temp_Kurve[temp_Kurve > GZG['nSamples'][c]] = []

        for n in range(1, len(temp_Kurve) + 1):
            nEV = GZG['Kurve'][n]
            for i in range(1, GZG['N_MC'] + 1):
                b = np.random.randint(1, GZG['N_Pool'], nEV)

                temp_GG_norm[i] = np.max(np.sum(Pool_MC[includeTime, b], axis=1, where=~np.isnan(Pool_MC))) / nEV

            # Quantile computations...
            GZG['MW'][n, c] = np.mean(temp_GG_norm)
            GZG['MW_Q90'][n, c] = np.percentile(temp_GG_norm, 90, interpolation='lower')
            GZG['MW_Q95'][n, c] = np.percentile(temp_GG_norm, 95, interpolation='lower')
            GZG['MW_Q99'][n, c] = np.percentile(temp_GG_norm, 99, interpolation='lower')
            GZG['MW_Qmax'][n, c] = np.max(temp_GG_norm)
            GZG['MW_Qmed'][n, c] = np.median(temp_GG_norm)

            if somePlots:
                # Scatter plot...
                pass

        print(f'{c}/6 - ', end='')

    except Exception as e:
        print(e)

print(' okay')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Alle EM mit BS durchgehen
fltr = np.where(Pool_inst_BS > 0)[0]
zr_bs_load = np.zeros((T, len(fltr)))
zr_bs_gen = np.zeros((T, len(fltr)))
zr_bs_gen_pv = np.zeros((T, len(fltr)))

for i in range(len(fltr)):
    zr_bs_load[:, i] = Pool_BS[:, fltr[i]] * (Pool_all[:, fltr[i]] > 0)
    zr_bs_gen[:, i] = Pool_BS[:, fltr[i]] * (Pool_all[:, fltr[i]] < 0)
    zr_bs_gen_pv[:, i] = (Pool_pv[:, fltr[i]] + Pool_BS[:, fltr[i]]) * (Pool_all[:, fltr[i]] < 0)

zr_bs_load[zr_bs_load < 0] = 0
zr_bs_gen[zr_bs_gen > 0] = 0
zr_bs_gen_pv[zr_bs_gen_pv > 0] = 0

zr_bs_load_scal = np.sum(zr_bs_load, axis=1) / np.sum(Pool_inst_BS[fltr])
zr_bs_gen_scal = np.sum(zr_bs_gen, axis=1) / np.sum(Pool_inst_BS[fltr])
zr_bs_gen_pv_scal = np.sum(zr_bs_gen_pv, axis=1) / np.sum(Pool_inst_PV[fltr])

# Plotting examples
plt.figure()
ts = np.array([datetime.timedelta(minutes=15) * i for i in range(366*96)])
plt.plot(ts, np.sum(zr_bs_load_scal, axis=1))
plt.plot(ts, np.sum(zr_bs_gen_scal, axis=1))
plt.plot(ts, np.sum(zr_bs_load_scal, axis=1) + np.sum(zr_bs_gen_scal, axis=1))
plt.show()

plt.figure()
x = np.arange(1, 366*96 + 1) / 4
plt.plot(x, np.sort(np.sum(zr_bs_load_scal, axis=1) + np.sum(zr_bs_gen_scal, axis=1))[::-1], linewidth=1.5)
plt.title("Duration curve BESS - Scenario 1a")
plt.grid(True)
plt.xlabel('hours')
plt.ylabel('concurrency')
plt.xlim(0, 8770)
plt.show()

plt.figure()
x = np.arange(1, 366*96 + 1) / 4
plt.plot(x, np.sort(zr_bs_gen_pv_scal)[::-1], linewidth=1.5)
plt.title("Duration curve BESS - Scenario 1a")
plt.grid(True)
plt.xlabel('hours')
plt.ylabel('concurrency')
plt.xlim(0, 8760)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import datetime

# Alle EM mit EV durchgehen
fltr = np.where(Pool_inst_EV > 0)[0]

a = np.sum(Pool_ev2[:, fltr], axis=1)
a2 = a / np.sum(Pool_inst_EV[fltr])

ts = np.array([datetime.timedelta(minutes=15) * i for i in range(366*96)])
plt.figure()
x = np.arange(1, 366*96 + 1) / 4
plt.plot(x, np.sort(a2)[::-1], linewidth=1.5)
plt.title("Duration curve EV - Scenario 1a")
plt.grid(True)
plt.xlabel('hours')
plt.ylabel('concurrency')
plt.xlim(0, 366*24)
plt.show()

# Alle EM mit EV durchgehen
fltr = np.where(Pool_inst_EV > 0)[0]
a_load = Pool_ev2[:, fltr].copy()
a_load[Pool_EM_pSum[fltr] < 0] = 0
a_load = np.sum(a_load, axis=1) / np.sum(Pool_inst_EV)
a_load[a_load < 0] = 0
a_gen = Pool_ev2[:, fltr].copy()
a_gen[Pool_EM_pSum[fltr] > 0] = 0
a_gen = np.sum(a_gen, axis=1) / np.sum(Pool_inst_EV)
a_gen[a_gen > 0] = 0

y = a_load + a_gen
GZG_V2G_all = y

if False:  # Change to True to plot
    plt.figure()
    x = np.arange(1, 366*96 + 1) / 4
    plt.plot(x, np.sort(y)[::-1], linewidth=1.5)
    plt.title("Duration curve EV - Scenario 1a")
    plt.grid(True)
    plt.xlabel('hours')
    plt.ylabel('concurrency')
    plt.xlim(0, 366*24)
    plt.show()


In [ ]:
import scipy.io as sio

# Zusammenfassung der GZG-Kurve nach Stichprobenzahl und installierter Leistung
weightFactor = GZG_pInstTotal / np.sum(GZG_pInstTotal)
GZG_GZG_weighted = np.sum(GZG_MW_Q95 * np.tile(weightFactor.reshape(-1, 1), (1, GZG_N_Kurve)), axis=1)
GZG_maxFactorBS = np.max(zr_bs_load_scal)
GZG_timeSeriesBSLoadsScal = zr_bs_load_scal

# Save the data
file_name = f"{folderResForStefan}/concurrency_v3_{scenario}_{country_code}.mat"
sio.savemat(file_name, {'GZG_GZG_weighted': GZG_GZG_weighted, 'GZG_maxFactorBS': GZG_maxFactorBS, 'GZG_timeSeriesBSLoadsScal': GZG_timeSeriesBSLoadsScal})

print('Success')
success = True
